# Face Clustering for Photo Albums


## Imports

In [11]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.core.display import display
from ipywidgets import interact, interact_manual
from ipywidgets import Button, HBox, VBox, GridspecLayout
import cv2 
from matplotlib import pyplot as plt
import src.main as main




In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999; // prevents collapsing of the image preview

<IPython.core.display.Javascript object>

##  Select File Path

In [3]:
image_folder = r"C:\Users\JZH\PycharmProjects\face_recognition\code\dat\AndreasAlbums\A1";

## Extract Faces from Album / Image(s)

In [4]:

radio = widgets.RadioButtons(
    options=['Scanned Album', 'Image(s)'],
    description='Mode:',
    disabled=False
)
display(radio)

RadioButtons(description='Mode:', options=('Scanned Album', 'Image(s)'), value='Scanned Album')

In [5]:
isAlbum = True;
main.extract_faces_from_folder(image_folder, isAlbum);

## Get Face Embeddings

In [6]:
embeddings, face_imgs = main.get_embeddings_and_imgs(image_folder);

In [50]:
threshold_slider = widgets.IntSlider(
    value=30,
    min=1,
    max=100,
    step=1,
    description='Threshold:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)
@interact
def cluster(threshold = threshold_slider):
    print("Class for every Face (-1 stands for undetected)\n")
    predictions, cluster_count, avg_imgs_per_cluster = main.get_clusters(embeddings,threshold)
    print("No of Clusters: " +  "\x1b[31m "+str(cluster_count)+ "\x1b[0m")
    print("Avg Images per Cluster: " +  "\x1b[31m "+str(avg_imgs_per_cluster)+ "\x1b[0m" )    
    print("\n Clusters:")
    print(predictions)
    
preview_button =widgets.Button(
    description='Preview anzeigen',
    disabled=False,
    button_style='info', 
    tooltip='Zeigt Preview für die generierten Cluster an',
    icon='image' # (FontAwesome names without the `fa-` prefix)
)


out = widgets.Output(
    max_height = 100

)
def show_preview(b):
    threshold = threshold_slider.value
    predictions, cluster_count, avg_imgs_per_cluster = main.get_clusters(embeddings,threshold)
    min_cluster_size  = 2
    max_images_per_plot = 3
    labels = np.unique(predictions)
    labels = labels[labels > -1]
    checkboxes = []
    all_clusters_box = []
    for idx, label in enumerate(labels):
        image_indices = np.where(predictions == label)[0]
        if image_indices.shape[0] >= min_cluster_size:
            cluster_box = []
            grid = GridspecLayout(1, 7, layout = {'overflow':'hidden'})
            columns = min(image_indices.shape[0], max_images_per_plot)
            for i in range(columns):
                file = open(face_imgs[image_indices[i]], "rb")
                image = file.read()
                img = widgets.Image(
                    value= image,
                    format='png',
                    width=200,
                    height=200,
                )
                grid[0,i] = img 
                
            cbox = widgets.Checkbox(False, description='Link')
            grid[0,3] = cbox
            
            for i in range(columns):
                file = open(face_imgs[image_indices[i]], "rb")
                image = file.read()
                img = widgets.Image(
                    value= image,
                    format='png',
                    width=200,
                    height=200,
                )
                grid[0,4+i] = img 
            name = widgets.Text(
                value=str(idx),
                placeholder='Enter Name',
                description='Name:',
                disabled=False
            )
            cluster_box.append(grid)
            cluster_box.append(name)
            all_clusters_box.append(VBox(cluster_box))
    display(VBox(all_clusters_box))

        
        
preview_button.on_click(show_preview)

      
display(preview_button)


interactive(children=(IntSlider(value=30, continuous_update=False, description='Threshold:', min=1), Output())…

Button(button_style='info', description='Preview anzeigen', icon='image', style=ButtonStyle(), tooltip='Zeigt …

In [ ]:
%qtconsole

In [ ]:
def show_preview(b):
    with out:
        out.clear_output()
        threshold = threshold_slider.value
        predictions, cluster_count, avg_imgs_per_cluster = main.get_clusters(embeddings,threshold)
        min_cluster_size  = 2
        max_images_per_plot = 3
        labels = np.unique(predictions)
        labels = labels[labels > -1]
        checkboxes = []
        for idx, label in enumerate(labels):
            image_indices = np.where(predictions == label)[0]
            if image_indices.shape[0] >= min_cluster_size:
                fig = plt.figure()
                rows = 1
                columns = min(image_indices.shape[0], max_images_per_plot)
                ax = []
                for i in range(columns * rows):
                    image_indice = image_indices[i]
                    img = face_imgs[image_indice].astype(int)
                    ax.append(fig.add_subplot(rows, columns, i + 1))
                    curr_ax = ax[-1]
                    curr_ax.set_xticks([])
                    curr_ax.set_yticks([])
                    plt.imshow(img)
            # use ax for further manipulations if necessary
            name = widgets.Text(
                value=str(idx),
                placeholder='Enter Name',
                description='Name:',
                disabled=False
            )
            display(name)
            plt.show()
            checkbox = widgets.Checkbox(
                value=True,
                description='Correct?',
                disabled=False
            )
            checkboxes.append(checkbox) 
            display(checkbox)